In [65]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [85]:
player_file = "resources/player_data.csv"
player_df = pd.read_csv(player_file)
player_df.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [86]:
season_file = "resources/seasons_stats.csv"
season_df = pd.read_csv(season_file)
del season_df['Unnamed: 0']
season_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


### Players DataFrame

In [87]:
# filter on columns we care about, and rename column "Position"
new_players_df = player_df[["name", "year_start", "year_end", "position", "college"]]
new_players_df = new_players_df.rename(columns={"name": "Player", "position": "player_position"})
new_players_df.head()

,Player,year_start,year_end,player_position,college
0,Alaa Abdelnaby,1991,1995,F-C,Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,San Jose State University


In [88]:
# # Create a filtered dataframe from specific columns
# premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
# premise_transformed= premise_df[premise_cols].copy()

# # Rename the column headers
# premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
#                                                           "Premises Name": "premise_name",
#                                                           "County ID Code": "county_id"})

# # Clean the data by dropping duplicates and setting the index
# premise_transformed.drop_duplicates("id", inplace=True)
# premise_transformed.set_index("id", inplace=True)

# premise_transformed.head()

### Season Stats DataFrame

In [89]:
# filter on columns we care about and rename column for No. of Games, and Position
new_season_df = season_df[["Player", "Pos", "Tm", "G", "GS", "Year", "PER", "AST", "TRB", "PTS"]]
new_season_df = new_season_df.rename(columns={"Tm": "Team", "G": "num_of_games", "Pos": "player_position", "Year": "Season"})
new_season_df.head()

,Player,player_position,Team,num_of_games,GS,Season,PER,AST,TRB,PTS
0,Curly Armstrong,G-F,FTW,63.0,NaN,1950.0,NaN,176.0,NaN,458.0
1,Cliff Barker,SG,INO,49.0,NaN,1950.0,NaN,109.0,NaN,279.0
2,Leo Barnhorst,SF,CHS,67.0,NaN,1950.0,NaN,140.0,NaN,438.0
3,Ed Bartels,F,TOT,15.0,NaN,1950.0,NaN,20.0,NaN,63.0
4,Ed Bartels,F,DNN,13.0,NaN,1950.0,NaN,20.0,NaN,59.0


In [81]:
# county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
# county_transformed = county_df[county_cols].copy()

# # Rename the column headers
# county_transformed = county_transformed.rename(columns={"ID": "id",
#                                                          "County Name (Licensee)": "county_name",
#                                                          "License Count": "license_count",
#                                                          "County ID Code": "county_id"})

# # Set index
# county_transformed.set_index("id", inplace=True)

# county_transformed.head()

### Create database connection

In [90]:
connection_string = "postgres:postgres@localhost:5432/players"
engine = create_engine(f'postgresql://{connection_string}')

In [91]:
# Confirm tables
engine.table_names()

['players_data', 'seasons_stats']

### Load DataFrames into database

In [92]:
new_players_df.to_sql(name='players_data', con=engine, if_exists='append', index=True)

In [93]:
new_season_df.to_sql(name='seasons_stats', con=engine, if_exists='append', index=True)